### Study of effect of national factors on home prices in US

**Task -** Using publically available data for the national factors that impact supply and demand of homes in US, build a model to study the effect of these variables on home prices.

**Approach -** The following variables are chosen for the study-
- Unemployment Rate
- Per Capita GDP
- Median Household Income
- Construction Prices
- CPI 
- Interest Rates
- Number of new houses supplied
- Working Population
- Percentage of population above 65
- Housing subsidies
- Number of Households

As a proxy to the home prices, S&P CASE-SHILLER Index is used. 

All of the data is downloaded from [https://fred.stlouisfed.org/].

As Data for the year 2023 and 2024 was not available for these factors :
- Percentage of population above 65
- Housing subsidies
- Number of households 
- Median Household income 

So ,the data is taken from years 2002-2022.

Data for all the variables is downloaded, preprocessed and combined to create a datset. Data for different variables had different frequencies. So, to combine the data, necessary interpolations are made.

Linear Regression is used as most of the variables have high correlation with the target variable.

In [1]:
# importing libraries
import numpy as np
import pandas as pd

In [2]:
#Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("CSUSHPISA.csv")

#Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print(df_CS.shape)
df_CS.tail()

(241, 4)


,DATE,CSUSHPISA,Year,Month
236,2021-09-01,270.197,2021,9
237,2021-10-01,273.603,2021,10
238,2021-11-01,277.309,2021,11
239,2021-12-01,281.506,2021,12
240,2022-01-01,286.173,2022,1


In [3]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
print(df_unemp.shape)
df_unemp.tail()

(241, 2)


,DATE,UNRATE
236,2021-09-01,4.7
237,2021-10-01,4.5
238,2021-11-01,4.1
239,2021-12-01,3.9
240,2022-01-01,4.0


In [4]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("gdp_per_capita.csv", names = ["DATE", "Per_Capita_GDP"], skiprows = 1)
print(df_pcgdp.shape)
df_pcgdp.tail()

(81, 2)


,DATE,Per_Capita_GDP
76,2021-01-01,63224.0
77,2021-04-01,64153.0
78,2021-07-01,64611.0
79,2021-10-01,65648.0
80,2022-01-01,65284.0


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.

In [5]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS.csv")
print(df_Fed_rate.shape)
df_Fed_rate.tail()

(241, 2)


,DATE,FEDFUNDS
236,2021-09-01,0.08
237,2021-10-01,0.08
238,2021-11-01,0.08
239,2021-12-01,0.08
240,2022-01-01,0.08


In [6]:
#Construction Price Index
df_cons_price_index = pd.read_csv("Construction_price.csv", names = ["DATE", "Cons_Materials"],skiprows=1)
print(df_cons_price_index.shape)
df_cons_price_index.tail()

(241, 2)


,DATE,Cons_Materials
236,2021-09-01,317.136
237,2021-10-01,322.120
238,2021-11-01,328.940
239,2021-12-01,335.032
240,2022-01-01,345.742


In [7]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["DATE", "CPI"], skiprows = 1)
print(df_CPI.shape)
df_CPI.tail()   

(241, 2)


,DATE,CPI
236,2021-09-01,273.887
237,2021-10-01,276.434
238,2021-11-01,278.799
239,2021-12-01,280.808
240,2022-01-01,282.390


In [8]:
# Monthly new house supply
df_house = pd.read_csv("Monthly_house_supply.csv", names = ["DATE", "Houses"], skiprows = 1)
print(df_house.shape)
df_house.tail()

(241, 2)


,DATE,Houses
236,2021-09-01,6.2
237,2021-10-01,6.7
238,2021-11-01,5.9
239,2021-12-01,5.6
240,2022-01-01,5.9


In [9]:
# Population above 65

df_oldpop = pd.read_csv("oldpop.csv", names = ["DATE", "old_percent"], skiprows = 1)
print(df_oldpop.shape)
df_oldpop.tail()

(21, 2)


,DATE,old_percent
16,2018-01-01,15.397698
17,2019-01-01,15.791801
18,2020-01-01,16.223400
19,2021-01-01,16.678895
20,2022-01-01,17.128121


In [10]:
# Housing Subsidies

df_subsidy = pd.read_csv("Housing_subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print(df_subsidy.shape)
df_subsidy.tail()

(21, 2)


,DATE,Subsidy
16,2018-01-01,38.859
17,2019-01-01,40.185
18,2020-01-01,44.147
19,2021-01-01,45.299
20,2022-01-01,48.021


In [11]:
# Working age population

df_working = pd.read_csv("Working_age.csv", names = ["DATE", "Working_Population"], skiprows = 1)
print(df_working.shape)
df_working.tail()

(241, 2)


,DATE,Working_Population
236,2021-09-01,205475500.0
237,2021-10-01,205337300.0
238,2021-11-01,205361800.0
239,2021-12-01,205323500.0
240,2022-01-01,207063100.0


In [12]:
# Real Median Household Income

df_income = pd.read_csv("Median_income.csv", names = ["DATE", "Income"], skiprows = 1)
print(df_income.shape)
df_income.tail()

(21, 2)


,DATE,Income
16,2018-01-01,73030
17,2019-01-01,78250
18,2020-01-01,76660
19,2021-01-01,76330
20,2022-01-01,74580


In [13]:
# Number of households

df_households = pd.read_csv("Households.csv", names = ["DATE", "Num_Households"], skiprows = 1)
print(df_households.shape)
df_households.tail()

(21, 2)


,DATE,Num_Households
16,2018-01-01,127586.0
17,2019-01-01,128579.0
18,2020-01-01,128451.0
19,2021-01-01,129224.0
20,2022-01-01,131202.0


In [14]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()

,DATE,CSUSHPISA,Year,Month,Per_Capita_GDP
0,2002-01-01,117.143,2002,1,50091.0
1,2002-02-01,117.844,2002,2,NaN
2,2002-03-01,118.687,2002,3,NaN
3,2002-04-01,119.610,2002,4,50286.0
4,2002-05-01,120.724,2002,5,NaN


In [15]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(241, 10)


,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS
DATE,,,,,,,,,,
2002-01-01,117.143,2002,1,50091.0,182669300.0,4.2,177.7,5.7,142.0,1.73
2002-02-01,117.844,2002,2,NaN,182823900.0,4.0,178.0,5.7,142.2,1.74
2002-03-01,118.687,2002,3,NaN,183081600.0,4.1,178.5,5.7,143.2,1.73
2002-04-01,119.610,2002,4,50286.0,183315500.0,4.3,179.3,5.9,143.5,1.75
2002-05-01,120.724,2002,5,NaN,183462600.0,4.0,179.5,5.8,143.8,1.75


In [16]:
# Merging other dataframes 
others = [ df_households, df_income, df_subsidy, df_oldpop] #add df_urban
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,
2002-01-01,117.143,2002,1,50091.0,182669300.0,4.2,177.7,5.7,142.0,1.73,109297.0,65820,24.183,12.287458
2002-02-01,117.844,2002,2,NaN,182823900.0,4.0,178.0,5.7,142.2,1.74,109297.0,65820,24.183,12.287458
2002-03-01,118.687,2002,3,NaN,183081600.0,4.1,178.5,5.7,143.2,1.73,109297.0,65820,24.183,12.287458
2002-04-01,119.610,2002,4,50286.0,183315500.0,4.3,179.3,5.9,143.5,1.75,109297.0,65820,24.183,12.287458
2002-05-01,120.724,2002,5,NaN,183462600.0,4.0,179.5,5.8,143.8,1.75,109297.0,65820,24.183,12.287458


In [17]:
df.isna().sum()

CSUSHPISA               0
Year                    0
Month                   0
Per_Capita_GDP        160
Working_Population      0
Houses                  0
CPI                     0
UNRATE                  0
Cons_Materials          0
FEDFUNDS                0
Num_Households          0
Income                  0
Subsidy                 0
old_percent             0
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. We will fill the missing values in the "Per_Capita_GDP" column using linear interpolation. 

In [18]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [19]:
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,
2002-01-01,117.143,2002,1,50091.000000,182669300.0,4.2,177.7,5.7,142.0,1.73,109297.0,65820,24.183,12.287458
2002-02-01,117.844,2002,2,50156.000000,182823900.0,4.0,178.0,5.7,142.2,1.74,109297.0,65820,24.183,12.287458
2002-03-01,118.687,2002,3,50221.000000,183081600.0,4.1,178.5,5.7,143.2,1.73,109297.0,65820,24.183,12.287458
2002-04-01,119.610,2002,4,50286.000000,183315500.0,4.3,179.3,5.9,143.5,1.75,109297.0,65820,24.183,12.287458
2002-05-01,120.724,2002,5,50311.333333,183462600.0,4.0,179.5,5.8,143.8,1.75,109297.0,65820,24.183,12.287458


In [20]:
df.isna().sum()

CSUSHPISA             0
Year                  0
Month                 0
Per_Capita_GDP        0
Working_Population    0
Houses                0
CPI                   0
UNRATE                0
Cons_Materials        0
FEDFUNDS              0
Num_Households        0
Income                0
Subsidy               0
old_percent           0
dtype: int64

In [21]:
df.shape

(241, 14)

In [22]:
df.tail()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,
2021-09-01,270.197,2021,9,65302.333333,205475500.0,6.2,273.887,4.7,317.136,0.08,129224.0,76330,45.299,16.678895
2021-10-01,273.603,2021,10,65648.000000,205337300.0,6.7,276.434,4.5,322.120,0.08,129224.0,76330,45.299,16.678895
2021-11-01,277.309,2021,11,65526.666667,205361800.0,5.9,278.799,4.1,328.940,0.08,129224.0,76330,45.299,16.678895
2021-12-01,281.506,2021,12,65405.333333,205323500.0,5.6,280.808,3.9,335.032,0.08,129224.0,76330,45.299,16.678895
2022-01-01,286.173,2022,1,65284.000000,207063100.0,5.9,282.390,4.0,345.742,0.08,131202.0,74580,48.021,17.128121


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".

In [23]:
df.to_csv("prepared_dataset.csv")

In [24]:
df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,
2002-01-01,117.143,2002,1,50091.000000,182669300.0,4.2,177.7,5.7,142.0,1.73,109297.0,65820,24.183,12.287458
2002-02-01,117.844,2002,2,50156.000000,182823900.0,4.0,178.0,5.7,142.2,1.74,109297.0,65820,24.183,12.287458
2002-03-01,118.687,2002,3,50221.000000,183081600.0,4.1,178.5,5.7,143.2,1.73,109297.0,65820,24.183,12.287458
2002-04-01,119.610,2002,4,50286.000000,183315500.0,4.3,179.3,5.9,143.5,1.75,109297.0,65820,24.183,12.287458
2002-05-01,120.724,2002,5,50311.333333,183462600.0,4.0,179.5,5.8,143.8,1.75,109297.0,65820,24.183,12.287458
